<a href="https://colab.research.google.com/github/DanZter/BERT_framework/blob/master/BERT_on_steroids_X_multicore_tpu_training_(time).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [19]:
import os
assert os.environ['COLAB_TPU_ADDR'], 'Make sure to select TPU from Edit > Notebook settings > Hardware accelerator'

In [20]:
pip install torch===1.5.1 torchvision===0.6.1 -f https://download.pytorch.org/whl/torch_stable.html

Looking in links: https://download.pytorch.org/whl/torch_stable.html
  Using cached https://files.pythonhosted.org/packages/62/01/457b49d790b6c4b9720e6f9dbbb617692f6ce8afdaadf425c055c41a7416/torch-1.5.1-cp36-cp36m-manylinux1_x86_64.whl
  Using cached https://files.pythonhosted.org/packages/9a/f1/535a407b4a265adf2dd7c2c2458217e37c5fe83ec97234e66c564592a9a0/torchvision-0.6.1-cp36-cp36m-manylinux1_x86_64.whl
  Found existing installation: torch 1.5.0a0+ab660ae
    Uninstalling torch-1.5.0a0+ab660ae:
      Successfully uninstalled torch-1.5.0a0+ab660ae
  Found existing installation: torchvision 0.6.0a0+3c254fb
    Uninstalling torchvision-0.6.0a0+3c254fb:
      Successfully uninstalled torchvision-0.6.0a0+3c254fb


In [21]:
VERSION = "1.5"  #@param ["1.5" , "20200325", "nightly"]
!curl https://raw.githubusercontent.com/pytorch/xla/master/contrib/scripts/env-setup.py -o pytorch-xla-env-setup.py
!python pytorch-xla-env-setup.py --version $VERSION

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  4994  100  4994    0     0  27591      0 --:--:-- --:--:-- --:--:-- 27591
Updating... This may take around 2 minutes.
Updating TPU runtime to pytorch-1.5 ...
Uninstalling torch-1.5.1:
  Successfully uninstalled torch-1.5.1
Uninstalling torchvision-0.6.1:
  Successfully uninstalled torchvision-0.6.1
Copying gs://tpu-pytorch/wheels/torch-1.5-cp36-cp36m-linux_x86_64.whl...
- [1 files][ 79.0 MiB/ 79.0 MiB]                                                
Operation completed over 1 objects/79.0 MiB.                                     
Copying gs://tpu-pytorch/wheels/torch_xla-1.5-cp36-cp36m-linux_x86_64.whl...
- [1 files][106.6 MiB/106.6 MiB]                                                
Operation completed over 1 objects/106.6 MiB.                                    
Copying gs://tpu-pytorch/wheels/torchvision-1.5-cp36-cp36m-li

In [22]:
# imports pytorch
import torch

# imports the torch_xla package
import torch_xla
import torch_xla.core.xla_model as xm
import torch_xla.distributed.xla_multiprocessing as xmp
import torch_xla.distributed.parallel_loader as pl

In [23]:
%tensorflow_version 2.x
import tensorflow as tf
print("Tensorflow version " + tf.__version__)

try:
  tpu = tf.distribute.cluster_resolver.TPUClusterResolver()  # TPU detection
  print('Running on TPU ', tpu.cluster_spec().as_dict()['worker'])
except ValueError:
  raise BaseException('ERROR: Not connected to a TPU runtime; please see the previous cell in this notebook for instructions!')

tf.config.experimental_connect_to_cluster(tpu)
# tf.tpu.experimental.initialize_tpu_system(tpu)
tpu_strategy = tf.distribute.experimental.TPUStrategy(tpu)

Tensorflow version 2.2.0
Running on TPU  ['10.90.29.234:8470']
INFO:tensorflow:Found TPU system:


INFO:tensorflow:Found TPU system:


INFO:tensorflow:*** Num TPU Cores: 8


INFO:tensorflow:*** Num TPU Cores: 8


INFO:tensorflow:*** Num TPU Workers: 1


...

INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU:7, TPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


In [24]:
!pip install transformers

In [25]:
import pandas as pd
import numpy as np
import transformers
from transformers import AdamW, get_linear_schedule_with_warmup
import torch.nn as nn
from sklearn import model_selection
from scipy import stats
from tqdm import tqdm

import warnings
warnings.filterwarnings("ignore")
# import logging
# logging.basicConfig(level=logging.ERROR)

In [28]:
class BERTBaseUncased(nn.Module):
    def __init__(self, bert_path):
        super(BERTBaseUncased, self).__init__()
        self.bert_path = bert_path
        self.bert = transformers.BertModel.from_pretrained(self.bert_path)
        self.bert_drop = nn.Dropout(0.3)
        self.out = nn.Linear(768, 30)       # pool output has 768 features, 30 targets

    def forward(self, ids, mask, token_type_ids):
        _, o2 = self.bert(ids, attention_mask=mask, token_type_ids=token_type_ids)  # not using o1 - Sequential Output
        bo = self.bert_drop(o2)                                                     # using o2 - Pooled Output
        return self.out(bo)

class BERTDatasetTraining:
    def __init__(self, qtitle, qbody, answer, targets, tokenizer, max_len):
        self.qtitle = qtitle
        self.qbody = qbody
        self.answer = answer
        self.tokenizer = tokenizer
        self.max_len = max_len
        self.targets = targets                   # numpy array of size: no. of samples * 30(targets)

    def __len__(self):
        return len(self.answer)                  # returns length of title or body or answer

    def __getitem__(self, item):                 # takes in index and returns output
        question_title = str(self.qtitle[item])
        question_body = str(self.qbody[item])
        answer = str(self.answer[item])

        # [CLS] [Q-TITLE] [Q-BODY] [SEP] [ANSWER] [SEP]

        inputs = self.tokenizer.encode_plus(
            question_title + " " + question_body,
            answer,
            add_special_tokens=True,
            max_length=self.max_len,
            truncation=True 
        )

        ids = inputs["input_ids"]
        token_type_ids = inputs["token_type_ids"]
        mask = inputs["attention_mask"]

        padding_len = self.max_len - len(ids)
        ids = ids + ([0] * padding_len)
        token_type_ids = token_type_ids + ([0] * padding_len)
        mask = mask + ([0] * padding_len)

        return {
            "ids": torch.tensor(ids, dtype=torch.long),
            "token_type_ids": torch.tensor(token_type_ids, dtype=torch.long),
            "mask": torch.tensor(mask, dtype=torch.long),
            "targets": torch.tensor(self.targets[item, :], dtype=torch.float)
        }

def loss_fn(outputs, targets):
    return nn.BCEWithLogitsLoss()(outputs, targets)

def train_loop_fn(data_loader, model, optimizer, device, scheduler=None):
    model.train()
    for bi, d in enumerate(data_loader):
        ids = d["ids"]
        mask = d["mask"]
        token_type_ids = d["token_type_ids"]
        targets = d["targets"]

        ids = ids.to(device, dtype=torch.long)
        mask = mask.to(device, dtype=torch.long)
        token_type_ids = token_type_ids.to(device, dtype=torch.long)
        targets = targets.to(device, dtype=torch.float)

        optimizer.zero_grad()
        outputs = model(ids=ids, mask=mask, token_type_ids=token_type_ids)
        loss = loss_fn(outputs, targets)
        loss.backward()
        # optimizer.step()                                         # cuda
        xm.optimizer_step(optimizer)                              # tpu
        if scheduler is not None:
            scheduler.step()
        # if bi % 10 == 0:
        #     xm.master_print(f"batch_index={bi}, loss={loss}")

def eval_loop_fn(data_loader, model, device):
    model.eval()
    fin_targets = []
    fin_outputs = []
    for bi, d in enumerate(data_loader):
        ids = d["ids"]
        mask = d["mask"]
        token_type_ids = d["token_type_ids"]
        targets = d["targets"]

        ids = ids.to(device, dtype=torch.long)
        mask = mask.to(device, dtype=torch.long)
        token_type_ids = token_type_ids.to(device, dtype=torch.long)
        targets = targets.to(device, dtype=torch.float)

        outputs = model(ids=ids, mask=mask, token_type_ids=token_type_ids)
        loss = loss_fn(outputs, targets)

        fin_targets.append(targets.cpu().detach().numpy())
        fin_outputs.append(outputs.cpu().detach().numpy())        # Linear Layer: can apply sigmoid here

        return np.vstack(fin_outputs), np.vstack(fin_targets)

def run(index, flags):

    # MAX_LEN = 512
    # TRAIN_BATCH_SIZE = 4
    # EPOCHS = 20

    flags['TRAIN_BATCH_SIZE'] = 16
    flags['TEST_BATCH_SIZE'] = 16
    flags['MAX_LEN'] = 512
    flags['EPOCHS'] = 20
    flags['seed'] = 1234  
    torch.manual_seed(flags['seed'])

    dfx = pd.read_csv("/content/drive/My Drive/Colab Notebooks/input/google_quest_train.csv").fillna("none")
    df_train, df_valid = model_selection.train_test_split(dfx, random_state = 42, test_size = 0.1)
    df_train = df_train.reset_index(drop=True)
    df_valid = df_valid.reset_index(drop=True)

    sample = pd.read_csv("/content/drive/My Drive/Colab Notebooks/input/google_quest_sample_submission.csv")
    target_cols = list(sample.drop("qa_id", axis=1).columns)
    train_targets = df_train[target_cols].values
    valid_targets = df_valid[target_cols].values

    tokenizer =transformers.BertTokenizer.from_pretrained("/content/drive/My Drive/Colab Notebooks/input/bert_base_uncased")

    train_dataset = BERTDatasetTraining(
        qtitle=df_train.question_title.values,
        qbody=df_train.question_body.values,
        answer=df_train.answer.values,
        targets=train_targets,
        tokenizer=tokenizer,
        max_len=flags['MAX_LEN']
    )
    train_sampler = torch.utils.data.DistributedSampler(
       train_dataset,
       num_replicas=xm.xrt_world_size(),
       rank=xm.get_ordinal(),
       shuffle=True 
    )
    train_data_loader = torch.utils.data.DataLoader(
        train_dataset,
        batch_size= flags['TRAIN_BATCH_SIZE'],
        sampler=train_sampler
    )
    ############
    valid_dataset = BERTDatasetTraining(
        qtitle=df_valid.question_title.values,
        qbody=df_valid.question_body.values,
        answer=df_valid.answer.values,
        targets=valid_targets,
        tokenizer=tokenizer,
        max_len=flags['MAX_LEN']
    )
    valid_sampler = torch.utils.data.DistributedSampler(
       valid_dataset,
       num_replicas=xm.xrt_world_size(),
       rank=xm.get_ordinal()
    )
    valid_data_loader = torch.utils.data.DataLoader(
        valid_dataset,
        batch_size=flags['TEST_BATCH_SIZE'],
        sampler=valid_sampler
    )

    # device = "cuda"                   # cuda
    device = xm.xla_device()            # tpu
    lr = 3e-5 *xm.xrt_world_size()
    num_train_steps = int(len(train_dataset)/ flags['TRAIN_BATCH_SIZE'] / xm.xrt_world_size() * flags['EPOCHS'])
    model = BERTBaseUncased("/content/drive/My Drive/Colab Notebooks/input/bert_base_uncased").to(device)

    optimizer = AdamW(model.parameters(), lr=lr)
    scheduler = get_linear_schedule_with_warmup(
        optimizer,
        num_warmup_steps=0,
        num_training_steps=num_train_steps
    )
    for epoch in tqdm(range(flags['EPOCHS']), total=flags['EPOCHS']):
      para_loader = pl.ParallelLoader(train_data_loader, [device])
      train_loop_fn(para_loader.per_device_loader(device), model, optimizer, device, scheduler)

      para_loader = pl.ParallelLoader(valid_data_loader, [device])
      o, t = eval_loop_fn(para_loader.per_device_loader(device), model, device)

      spear = []
      for jj in range(t.shape[1]):
          p1 = list(t[:,jj])
          p2 = list(o[:, jj])
          coef, _ = np.nan_to_num(stats.spearmanr(p1,p2))
          spear.append(coef)
      spear = np.mean(spear)
      xm.master_print(f"epoch = {epoch}, spearman = {spear}")
      # torch.save(model.state_dict(), "model.bin")         # cuda
      xm.save(model.state_dict(), "/content/drive/My Drive/Colab Notebooks/models/bert_on_steroids_multi_tpu_model.bin")          # tpu

# if __name__=="__main__":
#   xmp.spawn(run, nprocs=1)

# # Spawns eight of the map functions, one for each of the eight cores on
# # the Cloud TPU
flags = {}
xmp.spawn(run, args=(flags,), nprocs=8, start_method='fork')

  0%|          | 0/20 [00:00<?, ?it/s]

epoch = 0, spearman = 0.14000572152720162


  5%|▌         | 1/20 [01:35<30:19, 95.74s/it]

epoch = 1, spearman = 0.2594959403731799


 10%|█         | 2/20 [02:32<25:13, 84.09s/it]

...

 90%|█████████ | 18/20 [17:08<01:48, 54.29s/it]

epoch = 18, spearman = 0.3494674659206098


 95%|█████████▌| 19/20 [18:06<00:55, 55.41s/it]

epoch = 19, spearman = 0.3575525847144326


100%|██████████| 20/20 [19:19<00:00, 57.98s/it]



100%|██████████| 20/20 [19:59<00:00, 59.96s/it]
